In [1]:
import requests

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [130]:
df_paper_authors = pd.read_csv('../data/author benchmarks - scopus_data.csv')
df_paper_authors['doi'] = df_paper_authors['doi'].str.lower()

In [131]:
df_paper_authors.doi.notna().sum()

3442

In [132]:
df_paper_authors

,scopus_author_id,scopus_last_name,scopus_first_name,scopus_title,doi,scopus_has_doi,author_doi_pair,openalex_main_author_id,match_openalex
0,14632830700,Waltman,Ludo,Retraction of Predatory publishing in Scopus: ...,10.1007/s11192-022-04565-6,1.0,"14632830700,10.1007/s11192-022-04565-6",A2140778852,0.0
1,14632830700,Waltman,Ludo,Funding COVID-19 research: Insights from an ex...,10.1162/qss_a_00212,1.0,"14632830700,10.1162/qss_a_00212",A2140778852,1.0
2,14632830700,Waltman,Ludo,The challenges of scientometric studies of pre...,10.1162/qss_e_00214,1.0,"14632830700,10.1162/qss_e_00214",A2140778852,1.0
3,14632830700,Waltman,Ludo,A large-scale bibliometric analysis of global ...,10.1007/s10584-022-03324-z,1.0,"14632830700,10.1007/s10584-022-03324-z",A2140778852,1.0
4,14632830700,Waltman,Ludo,"Innovating peer review, reconfiguring scholarl...",10.1108/jd-01-2022-0022,1.0,"14632830700,10.1108/JD-01-2022-0022",A2140778852,1.0
...,...,...,...,...,...,...,...,...,...
3986,7406904255,Zhang,Beichuan,UAV aided intelligent routing for ad-hoc wirel...,NaN,NaN,NaN,NaN,NaN
3987,35339039700,Hymel,Mona L.,Editors' preface,NaN,NaN,NaN,NaN,NaN
3988,35339039700,Hymel,Mona L.,Innovation addressing climate change challenge...,10.4337/9781788973366,NaN,NaN,NaN,NaN
3989,35339039700,Hymel,Mona L.,Fighting for water: The role of federal market...,10.4337/9781786431219,NaN,NaN,NaN,NaN


In [134]:
subset = df_paper_authors[df_paper_authors['openalex_main_author_id'].isna()]
dois = subset['doi'].dropna().unique()

endpoint = "works"
size = 50
loop_index = 0
works = []
for list_index in range(0, len(dois), size):
    subset = dois[list_index:list_index+size]
    if '10.1007/BF00773018' in subset:
        print('ddd')
    pipe_separated_ids = "|".join(subset)
    r = requests.get(f"https://api.openalex.org/{endpoint}?filter=doi:{pipe_separated_ids}&per-page={size}")
    results = r.json()['results']
    works.extend(results)
    loop_index += 1
print(f"collected {len(works)} works using {loop_index} api calls")

collected 186 works using 4 api calls


In [135]:
len(dois)

186

In [136]:
data = []
for work in works:
    doi = work['doi'].replace('https://doi.org/', '').lower() if work['doi'] else None
    openalex_id = work['id'].replace('https://openalex.org/', '')
    for authorship in work['authorships']:
        author_id = authorship['author'].get('id', None)
        if author_id:
            author_id = author_id.replace('https://openalex.org/', '')

        data.append({
            'doi': doi,
            'openalex_work_id': openalex_id,
            'openalex_author_id': author_id,
            'openalex_author_display_name': authorship['author'].get('display_name', None),
            'openalex_author_affil': authorship['author'].get('last', None),
        })
df_alex_paper_authors = pd.DataFrame(data)

In [138]:
_df = df_paper_authors[['scopus_author_id', 'doi']].dropna(subset=['doi'])
_df = _df.merge(df_alex_paper_authors, how='right', on='doi')

In [139]:
modes = _df.groupby('scopus_author_id')[['openalex_author_id', 'openalex_author_display_name']].agg(pd.Series.mode)
modes.to_csv('../data/author_modes.csv')

In [141]:
_df[_df['scopus_author_id']==35339039700].sort_values('openalex_author_id')

,scopus_author_id,doi,openalex_work_id,openalex_author_id,openalex_author_display_name,openalex_author_affil
971,35339039700,10.4337/9781788973366,W2889824612,A1972506302,Hope Ashiabor,None
985,35339039700,10.4337/9781786431219,W2509870086,A1972506302,Hope Ashiabor,None
969,35339039700,10.4337/9781788973366,W2889824612,A2033509636,Larry Kreiser,None
982,35339039700,10.4337/9781786431219,W2509870086,A2033509636,Larry Kreiser,None
970,35339039700,10.4337/9781788973366,W2889824612,A2096851303,Janet E. Milne,None
984,35339039700,10.4337/9781786431219,W2509870086,A2096851303,Janet E. Milne,None
983,35339039700,10.4337/9781786431219,W2509870086,A2236736286,Bill Butcher,None
968,35339039700,10.4337/9781788973366,W2889824612,A2244336134,Mona L. Hymel,None
980,35339039700,10.1080/10192557.2009.11788216,W2187928683,A2244336134,Mona L. Hymel,None
979,35339039700,10.1080/10192557.2009.11788216,W2187928683,A2284751507,Peter J. Gillies,None


In [144]:
df_authors = pd.read_csv('../data/author benchmarks - authors.csv')

In [145]:
author_ids = df_authors['openalex_main_author_id'].dropna().unique()

endpoint = "authors"
size = 50
loop_index = 0
authors_alex = []
for list_index in range(0, len(author_ids), size):
    subset = author_ids[list_index:list_index+size]
    pipe_separated_ids = "|".join(subset)
    r = requests.get(f"https://api.openalex.org/{endpoint}?filter=openalex:{pipe_separated_ids}&per-page={size}")
    results = r.json()['results']
    authors_alex.extend(results)
    loop_index += 1
print(f"collected {len(authors_alex)} authors using {loop_index} api calls")

collected 38 authors using 1 api calls


In [146]:
data = []
for au in authors_alex:
    openalex_id = au['id'].replace('https://openalex.org/', '')
    display_name = au['display_name']
    h_index = au['summary_stats']['h_index']
    affil = au['last_known_institution'].get('display_name', None) if au['last_known_institution'] else None
    works_count = au['works_count']
    data.append({
        'openalex_main_author_id': openalex_id,
        'openalex_display_name': display_name,
        'openalex_num_papers': works_count,
        'openalex_affil': affil,
        'openalex_h_index': h_index,
    })

df_authors_alex_api = pd.DataFrame(data)

In [147]:
out = df_authors_alex_api.merge(df_authors[['scopus_author_id', 'openalex_main_author_id']], how='inner', on='openalex_main_author_id').sort_values('scopus_author_id')
out.to_csv('../data/df_authors_alex_api.csv', index=False)

In [148]:
def get_works_one_author(author_id):
    endpoint = 'works'
    cursor = '*'

    select = ",".join((
        'id',
        'title',
        'doi',
    ))

    # loop through pages
    works_alex = []
    loop_index = 0
    while cursor:
        
        # set cursor value and request page from OpenAlex
        url = f"https://api.openalex.org/{endpoint}?filter=author.id:{author_id}&cursor={cursor}&select={select}"
        page_with_results = requests.get(url).json()
        
        results = page_with_results['results']
        works_alex.extend(results)

        # update cursor to meta.next_cursor
        cursor = page_with_results['meta']['next_cursor']
        loop_index += 1
        # if loop_index in [5, 10, 20, 50, 100] or loop_index % 500 == 0:
        #     print(f'{loop_index} api requests made so far')
    print(f'for {author_id}: made {loop_index} api requests. collected {len(works_alex)} works')
    return works_alex

In [149]:
# author_ids = df_authors_alex_api.openalex_main_author_id.dropna().unique()
author_ids = [
'A2225201119',
'A2141093348',
'A1710357775',
'A2345815135',
'A2244336134',
]
# display_names = df_authors_alex_api.dropna(subset=['openalex_main_author_id']).set_index('openalex_main_author_id')['openalex_display_name']
data = []
for author_id in author_ids:
    this_author_works = get_works_one_author(author_id)
    for work in this_author_works:
        doi = work['doi'].replace('https://doi.org/', '') if work['doi'] else None
        openalex_work_id = work['id'].replace('https://openalex.org/', '')
        data.append({
            'openalex_author_id': author_id,
            # 'openalex_author_display_name': display_names[author_id],
            'doi': doi,
            'title': work['title'],
            'openalex_work_id': openalex_work_id,
        })
df_alex_paper_authors = pd.DataFrame(data)

for A2225201119: made 65 api requests. collected 1577 works
for A2141093348: made 3 api requests. collected 30 works
for A1710357775: made 4 api requests. collected 60 works
for A2345815135: made 9 api requests. collected 188 works
for A2244336134: made 2 api requests. collected 24 works


In [150]:
df_alex_paper_authors.to_csv('../data/df_alex_paper_authors2.csv')